# Libraries needed

In [2]:
import pandas as pd
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import *
import re
from random import randint

# Import data

In [191]:
df = pd.read_csv("""CSV File here""")
names = df['Name']
names = list(names)
df.shape

# Scrape page

In [ ]:
#Create Chrome driver element and pass url.
options = Options()
options.headless = False
driver = webdriver.Chrome(options = options)


#This function looks for an element (whcih you pass to it) and checks if it exists.
def check_exists_by_xpath(xpath):
    try:
        driver.find_element(By.XPATH,xpath)
    except NoSuchElementException:
        return False
    return True


#Create lists to store scraped info.
phones = []
postcodes = []
websites = []


#Loop over each name in list of company names.
for name in names:
    
    #Set random times for making the browsers wait. This minimises the risk of triggering CAPTCHA. 
    rand = randint(4,6)
    randloop = randint(6,9)

    #Go to search page.
    driver.get("""webpage which will be used for searching for companies""")
    
    time.sleep(randloop)

    #Look for the class.
    search_box = driver.find_element(By.XPATH, '//*[@id="globalsearch"]')

    #Take the company name from list and enter into search box.
    search_box.send_keys(name)

    #Submit the text.
    search_box.send_keys(Keys.RETURN)
    time.sleep(rand)

    #Click on search clubs button.
    xpath = "/html/body/div[1]/main/div/div/div[2]/div/div[1]/article/div[1]/div/div/a[1]"
    web_elem = driver.find_element(By.XPATH, xpath)
    driver.execute_script("arguments[0].click();", web_elem)
    time.sleep(rand)
    
    #Use custom funciton to check if page exists.
    page_exists = check_exists_by_xpath('//*[@id="clubs-table"]/tbody/tr/td[5]/div[1]/h4/a')
    
    
    if page_exists == True:

        #Click on club link and go to information page.
        xpath = '//*[@id="clubs-table"]/tbody/tr/td[5]/div[1]/h4/a'
        web_elem = driver.find_element(By.XPATH, xpath)
        driver.execute_script("arguments[0].click();", web_elem)
        time.sleep(rand)

        #Click on contact page.
        xpath = '//*[@id="ui-id-contact"]'
        web_elem = driver.find_element(By.XPATH, xpath)
        driver.execute_script("arguments[0].click();", web_elem)
        time.sleep(rand)

        #Use custom function to check if website link exists.
        web_exists = check_exists_by_xpath('//*[@id="contact"]/p/a')


        #Get website.
        if web_exists ==True:
            elem = driver.find_element(By.XPATH, '//*[@id="contact"]/p/a')
            website = elem.get_attribute('href')
        else:
            website = 'None'

        #Get text on page.
        info = driver.find_element(By.XPATH, '//*[@id="contact"]').text

        info = info.split('\n')


        # Get postcode element. 
        pcode = []

        for ps in info:

            if 'Postcode' in ps:

                pcode = ps

        postcode = pcode[10:]


        # Get phone number element. 
        pnum = []

        for ps in info:

            if 'Phone' in ps:

                pnum = ps

        phone = pnum[7:]

    #Fill with some null value if page does not exist.
    else:
        
        phone = 'None'
        postcode = 'None'
        website = 'None'
        
        
    phones.append(phone)
    postcodes.append(postcode)
    websites.append(website)

# Find email using regex and filter parameters

In [ ]:
df = pd.read_csv("""Pull in new info from previous scrape""")
websites = df['Website']
websites = list(websites)
df.shape

In [ ]:
# Create driver chrome driver element and pass url.
options = Options()
options.headless = False
driver = webdriver.Chrome(options = options)

#Store emails in new list.
address = []

#Loop over each of the websites just collected.
for web in websites:
    
    if web != 'None':

        driver.get(web)

        time.sleep(5)

        source = driver.page_source
        
        #This string is used to decode webpage and match instances which look like email addresses. 
        EMAIL_REGEX = """(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

        #Compare all text in page source against regex string and take values which match. 
        list_of_emails = []

        for re_match in re.finditer(EMAIL_REGEX, source):
            list_of_emails.append(re_match.group())


        #Remove duplicates in list. 
        list_of_emails = list(set(list_of_emails))

        #This will be used to find elements in the collected list which match some characters in the company name.
        web = web.strip('http://www.')
        web = web[:4]
        
        
        #Using the section of company name above and other qualifiers to filter email results further. 
        email = []

        for i in list_of_emails:
    
            if 'gmail' in i or'outlook' in i or 'hotmail' in i or 'yahoo' in i or 'info' in i or web in i:
        
                email.append(i)          
            
            else:
    
                email = list_of_emails
        
    #Setting null value if no records found.     
    else:
        
        address.append('None')
        
        

# Grabbing contact form links where emails where not stated

In [ ]:
df = pd.read_csv('for_contacts.csv')
links = df['Website']
links = list(links)
df.shape

In [ ]:
options = Options()
options.headless = False
driver = webdriver.Chrome(options = options)

#Collecting results in list. 
forms = []


for i in links:
    
    try:
    
        if i != 'None':
            
            
            driver.get(i)

            #Get all website page links.
            strings = []

            elems = driver.find_elements_by_xpath("//a[@href]")
            for elem in elems:
                strings.append(elem.get_attribute("href"))


            #Get only links with 'Contact' in the name.
            form = []

            for i in strings:
                if 'Contact' in i or 'contact' in i:
                    form.append(i)
                    
            #Remove duplicates from list.
            form = list(set(form))


            forms.append(form)
            print(form)

        else: 

            forms.append('None')
            
            
    except:
        
        forms.append('None')
        
        continue
        
    #Saving to CSV each loop. 
    df = pd.DataFrame({'Contact':forms})
    df.to_csv('forms6.csv')

# Tidying up results

### Remove additional items and just keep first record

In [34]:
df = pd.read_csv("""Websites collected""")
contacts = df['Contact']
contacts = list(contacts)

In [35]:
#Split up results where there are multiple instances.
cons = []

for con in contacts:
    con = con.split(',')[0]
    cons.append(con)
    

### Replace values

In [38]:
df = pd.DataFrame({'Contact':cons})

In [39]:
df.loc[df['Contact'].str.contains('@'), 'Contact'] = 'None'
df.loc[df['Contact'].str.contains('remove reccuring value'),'Contact'] = 'None'

In [41]:
#Final save
df.to_csv('final_contacts.csv')